In [5]:
from flask import Flask, jsonify, request, render_template
import joblib
from flask_bootstrap import Bootstrap

import re
import numpy as np
from tensorflow import keras
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

# app = Flask(__name__, template_folder='templates')
# bootstrap=Bootstrap(app)


#Fonctions et variables nécessaires à la prédiction : 
max_len   =  100
INDEX_FROM=3 

word_to_id = keras.datasets.imdb.get_word_index()
word_to_id = {k:(v+INDEX_FROM) for k,v in word_to_id.items()}
word_to_id["<PAD>"] = 0
word_to_id["<START>"] = 1
word_to_id["<UNK>"] = 2
word_to_id["<UNUSED>"] = 3


def predict_class(model, x_test):

    # On prédit sur les données de test
    y_hat = model.predict(x_test)

    # On tranforme les prédictions en labels
    i_pos = [i for i in range(len(y_hat)) if y_hat[i]>0]
    i_neg = [i for i in range(len(y_hat)) if y_hat[i]<=0]

    y_pred   = np.zeros(len(y_hat))
    y_pred[i_pos] = 1
    y_pred[i_neg] = 0
    return y_pred

def encode_txt(txt) :
    txt=re.sub(r"[^a-zA-Z'-]", ' ', txt)
    res=[ word_to_id.get(i.lower(), 2) for i in txt.split()]
    res=np.asarray(res)
    res=res.reshape((1,res.shape[0]))
    res=pad_sequences(res, maxlen=max_len, truncating='post')
    return res



# 0 - hate speech 1 - offensive language 2 - neither
Labels = ["negative comment", "positive comment"]
model = joblib.load("model/rnn_sentiment.joblib.z")

Keras model archive loading:
File Name                                             Modified             Size
config.json                                    2023-03-08 14:44:48         4521
metadata.json                                  2023-03-08 14:44:48           64
variables.h5                                   2023-03-08 14:44:48      5240288
Keras weights file (<HDF5 file "variables.h5" (mode r)>) loading:
...layers\bidirectional
......vars
...layers\bidirectional\backward_layer
......vars
...layers\bidirectional\backward_layer\cell
......vars
.........0
.........1
.........2
...layers\bidirectional\forward_layer
......vars
...layers\bidirectional\forward_layer\cell
......vars
.........0
.........1
.........2
...layers\bidirectional\layer
......vars
...layers\bidirectional\layer\cell
......vars
...layers\bidirectional_1
......vars
...layers\bidirectional_1\backward_layer
......vars
...layers\bidirectional_1\backward_layer\cell
......vars
.........0
.........1
.........2
...layers\

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          320000    
                                                                 
 bidirectional (Bidirectiona  (None, None, 128)        66048     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               41216     
 nal)                                                            
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 6

In [ ]:
texte="Good movie"
texte2=encode_txt(texte)
res=predict_class(model, texte2)
classif=Labels[int(res[0])]
print(classif)


# @app.route('/')
# def hello():
#     return render_template('index.html')

# @app.route('/predict', methods=['POST'])
# def predict():
#     # On récupère le champ namequery du template index
#     texte = request.form['namequery']

#     texte2=encode_txt(texte)
#     res=predict_class(model, texte2)

#     classif=Labels[int(res[0])]

    

#     # Render the response in the result.html template
#     return render_template('result.html', classif=classif)

# # Lancement de l'application Flask
# if __name__ == '__main__':
#     app.run(debug=True, port=5000)
    
    